In [1]:
import tensorflow as tf

In [2]:
blooms_model = tf.keras.models.load_model('/LAB/blooms/blooms_model.pkl')

2024-03-12 15:43:47.088563: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11124 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6
2024-03-12 15:43:47.089398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 90 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:af:00.0, compute capability: 8.6


In [3]:
!pip install transformers

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
# !pip install --upgrade jupyter ipywidgets

In [5]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension


In [13]:
# import numpy as np
# import pandas as pd
# from transformers import BertTokenizer
# import tensorflow as tf

# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# def prepare_data(input_text, tokenizer):
#     token = tokenizer.encode_plus(
#         input_text,
#         max_length=256,
#         truncation=True,
#         padding='max_length',
#         add_special_tokens=True,
#         return_tensors='tf'
#     )
#     return {
#         'input_ids': tf.cast(token.input_ids, tf.float64),
#         'attention_mask': tf.cast(token.attention_mask, tf.float64)
#     }

# def make_prediction(model, processed_data, classes=['remember', 'understand', 'apply', 'analyze', 'evaluate','create']):
#     probs = model.predict(processed_data)[0]
#     return classes[np.argmax(probs)], np.argmax(probs) + 1  

# def read_questions_from_csv(filename):
#     df = pd.read_csv(filename)
#     questions = df['Questions'].tolist()  
#     return df, questions

# csv_filename = '/LAB/blooms/question_daa.csv'
# df, questions = read_questions_from_csv(csv_filename)

# predicted_blooms_levels = []
# predicted_blooms_encoded = []  
# for i, question in enumerate(questions):
#     processed_data = prepare_data(question, tokenizer)
#     result, encoded_value = make_prediction(blooms_model, processed_data=processed_data)
#     predicted_blooms_levels.append(result)
#     predicted_blooms_encoded.append(encoded_value)

# df['Predicted Blooms Level'] = predicted_blooms_levels
# df['Encoded Blooms Level'] = predicted_blooms_encoded  

# output_csv_filename = 'output.csv' 
# df[['Questions', 'Predicted Blooms Level', 'Encoded Blooms Level']].to_csv(output_csv_filename, index=False)

# print(f"Predicted blooms levels along with questions have been stored in {output_csv_filename}")


In [14]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['remember', 'understand', 'apply', 'analyze', 'evaluate','create']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)], np.argmax(probs) + 1  

def read_questions_from_csv(filename):
    df = pd.read_csv(filename)
    questions = df['Questions'].tolist()  
    return df, questions

def calculate_question_paper_quality(df):
    full_marks = df['Marks'].sum()  
    df['Encoded_Blooms_Level'] = df['Encoded_Blooms_Level'].astype(float)
    df['Question_Weightage'] = df['Marks'].astype(float)
    df['Question_Paper_Quality'] = (df['Encoded_Blooms_Level'] * df['Question_Weightage']) / full_marks
    return df['Question_Paper_Quality'].sum()




In [16]:
def categorize_difficulty(difficulty_score):
    if difficulty_score <= 1.5:
        return "Easy"
    elif 1.5 < difficulty_score <= 3.5:
        return "Medium"
    else:
        return "High"

csv_filename = '/LAB/blooms/final_output-daa.csv'
df, questions = read_questions_from_csv(csv_filename)

predicted_blooms_levels = []
predicted_blooms_encoded = [] 
for i, question in enumerate(questions):
    processed_data = prepare_data(question, tokenizer)
    result, encoded_value = make_prediction(blooms_model, processed_data=processed_data)
    predicted_blooms_levels.append(result)
    predicted_blooms_encoded.append(encoded_value)

df['Predicted_Blooms_Level'] = predicted_blooms_levels
df['Encoded_Blooms_Level'] = predicted_blooms_encoded  

output_csv_filename = 'output_final.csv'  
df[['Questions', 'Predicted_Blooms_Level', 'Encoded_Blooms_Level']].to_csv(output_csv_filename, index=False)

question_paper_quality = calculate_question_paper_quality(df)

difficulty_code = categorize_difficulty(question_paper_quality)

# df['Difficulty_Code'] = difficulty_code

df.to_csv(output_csv_filename, index=False)

print(f"Predicted blooms levels along with questions have been stored in {output_csv_filename}")
print(f"Score of the question paper is: {question_paper_quality}")
print(f"The difficulty  of the question paper is: {difficulty_code}")


Predicted blooms levels along with questions have been stored in output_final.csv
Score of the question paper is: 3.4444444444444438
The difficulty  of the question paper is: Medium
